#### CSCE 670 :: Information Storage & Retrieval :: Texas A&M University :: Spring 2023


# Project 2: Implicit Recommender Systems

# Part 1. Recommendations with Implicit Feedback 

For this part, we're going to build a simple **non-personalized** implicit recommendation algorithm. Since feedback like user clicks, purchases, and views is much more widespread than explicit ratings, implicit recommenders offer great opportunities for far-reaching impact. 

Concretely, the task of implicit recommendation is to recommend items to users based on implicit signals from users, i.e., we only know what items a user is interested in, but have no idea what items the user dislikes. So for this case, the dataset we could use for this implicit recommendation experiment only contains binary data with 1 representing that the user likes the item, and with 0 representing that we don't know the user's preference towards the item. Because of this, we cannot use the same evaluation method as explicit recommendation. Instead, we need to evaluate the implicit recommendation quality by a ranking task.

For this part, we will:
* load and process the MovieLens 1M dataset,
* transfer the explicit dataset to implicit one,
* build a non-personalized implicit recommender, 
* and evaluate your recommender.

To start out, we need to prepare the data. We will use the MovieLens 1M data from https://grouplens.org/datasets/movielens/1m/ in this homework. Lucky for you, we are providing the file containing the ratings -- ratings.dat  -- so all you need to do is load the ratings.dat file in the notebook as a DataFrame variable using the Pandas library. The code to do this has been provided in the next cell, but you need to run it. The resulting data variables are: train_mat is the numpy array variable for training data of size (#users, #items) with non-zero entries representing user-item ratings, and zero entries representing unknown user-item ratings; and test_mat is the numpy array variable for testing data of size (#users, #items).

In [56]:
import pandas as pd
import numpy as np
from scipy.sparse import coo_matrix

data_df = pd.read_csv('./ratings.dat', sep='::', names=["UserID", "MovieID", "Rating", "Timestamp"], engine='python')

# First, generate dictionaries for mapping old id to new id for users and movies
unique_MovieID = data_df['MovieID'].unique()
unique_UserID = data_df['UserID'].unique()
print(unique_MovieID.shape)
print(unique_UserID.shape)
j = 0
user_old2new_id_dict = dict()
for u in unique_UserID:
    user_old2new_id_dict[u] = j
    j += 1
j = 0

movie_old2new_id_dict = dict()
for i in unique_MovieID:
    movie_old2new_id_dict[i] = j
    j += 1

# Then, use the generated dictionaries to reindex UserID and MovieID in the data_df
user_list = data_df['UserID'].values
movie_list = data_df['MovieID'].values
for j in range(len(data_df)):
    user_list[j] = user_old2new_id_dict[user_list[j]]
    movie_list[j] = movie_old2new_id_dict[movie_list[j]]
data_df['UserID'] = user_list
data_df['movieID'] = movie_list

# generate train_df with 70% samples and test_df with 30% samples, and there should have no overlap between them.
np.random.seed(0)
train_index = np.random.random(len(data_df)) <= 0.7
train_df = data_df[train_index]
test_df = data_df[~train_index]
print(train_df)
print(test_df)
print(train_df.shape)
print(test_df.shape)

# generate train_mat and test_mat
num_user = len(data_df['UserID'].unique())
num_movie = len(data_df['MovieID'].unique())

train_mat = coo_matrix((train_df['Rating'].values, (train_df['UserID'].values, train_df['MovieID'].values)), shape=(num_user, num_movie)).astype(float).toarray()
test_mat = coo_matrix((test_df['Rating'].values, (test_df['UserID'].values, test_df['MovieID'].values)), shape=(num_user, num_movie)).astype(float).toarray()

(3706,)
(6040,)
         UserID  MovieID  Rating  Timestamp  movieID
0             0        0       5  978300760        0
2             0        2       3  978301968        2
3             0        3       4  978300275        3
4             0        4       5  978824291        4
5             0        5       3  978302268        5
...         ...      ...     ...        ...      ...
1000202    6039      336       4  956704996      336
1000204    6039      772       1  956716541      772
1000205    6039     1106       5  956704887     1106
1000207    6039      152       4  956715648      152
1000208    6039       26       4  956715569       26

[699628 rows x 5 columns]
         UserID  MovieID  Rating  Timestamp  movieID
1             0        1       3  978302109        1
7             0        7       5  978300719        7
8             0        8       4  978302268        8
10            0       10       5  978824268       10
13            0       13       4  978302124       13
...

It is very easy to transfer the explicit datasets you already generated to implicit ones: here, you will consider the watching behavior as the implicit signal showing that the user is interested in a movie. Thus, you can use the same train_df and test_df for implicit recommendation experiment with 'Rating' column ignored. And for train_mat and test_mat, you need to make all ratings to be value 1 and keep 0 entries the same.

In [57]:
# turn the explicit ratings to implicit feedback data
train_mat = (train_mat > 0).astype(float)
test_mat = (test_mat > 0).astype(float)
print(train_mat)
print(train_mat.shape)
print(test_mat.shape)
# print(test_mat)

[[1. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
(6040, 3706)
(6040, 3706)


## Part 1a: Building the non-personalized recommender 

In this part, you need to build a non-personalized recommendation model to provide a ranked list of 50 movies as the recommendation for each user. The model is very simple: for each user, the recommendation list is to rank the unwatched movies by their **popularity**, where the popularity is the number of implicit feedback each movie gets. In this case, although it is non-personalized recommender, the recommendation results may be different for users because the unwatched movies are different across users.

In the next cell, write your code to generate the ranked lists of movies by the popularity based recommendation algorithm for every user, store the result in a numpy array of size (#user, 50), entry (u, k) represents the movie id that is ranked at position k in the recommendation list to user u. Print out the top-5 recommended movies and their popularity for the first user (with id 0).

Hint: the popularity can only be calculated from train_mat, you cannot use test_mat here. 

In [58]:
# Generate a ranked list of movies by the popularity based recommendation algorithm. And print out the id and popularity of the top5 movies for the first user.
# Your Code Here...
def non_personalized_recommender(train_mat):
  popularity = train_mat.sum(axis=0).astype(int)
  print(popularity.shape)
  # print(np.sum(popularity))
  top_50 = np.zeros((num_user,50))
  

  for u in range(num_user):
    unwatched_movies = np.where(train_mat[u,:]== 0)[0]
    # print('\n',unwatched_movies)
    temp = np.zeros((num_movie))
    temp[unwatched_movies] = popularity[unwatched_movies]
    # print(temp)
    ranked_unwatched_movies = np.argsort(temp) 
    # print('\n',ranked_unwatched_movies)
    top_50_u = np.flip(ranked_unwatched_movies)[:50]
    # print(top_50_u)
    top_50[u,:] = top_50_u
  return top_50,popularity

top_50_list, popularity = non_personalized_recommender(train_mat)

print(top_50_list[0,:5])


print('\nTop 5 recommended movies for first user are:')
for i in range(5):
  print('Movie ID:', top_50_list[0,i], 'is at', i+1, 'position with popularity of', popularity[int(top_50_list[0,i])])


(3706,)
[104. 124.  64. 113.  97.]

Top 5 recommended movies for first user are:
Movie ID: 104.0 is at 1 position with popularity of 2423
Movie ID: 124.0 is at 2 position with popularity of 2086
Movie ID: 64.0 is at 3 position with popularity of 2064
Movie ID: 113.0 is at 4 position with popularity of 1862
Movie ID: 97.0 is at 5 position with popularity of 1845


## Part 1b: Evaluating the non-personalized recommender 

In this part, you need to evaluate your non-personalized recommendation by the held-out testing dataset test_mat for each user. For the implicit recommendation, two typical metrics are recall@k and precision@k. Here, you need to write the code to calculate recall@k and  precision@k for k=5, 20, 50 for each user, i.e., six metrics for every user. And please print out the average over all users for these six metrics.

Hint: if a user does not have any testing samples in test_mat, do not include this user in the final averaged metric.


In [59]:
# Calculate recall@k and precision@k with k=5, 20, 50 and print out the average over all users for these 9 metrics.
# Your Code Here...


def recall_func(relevant_retrieved_docs,total_relevant):
  return relevant_retrieved_docs/total_relevant

def precision_func(relevant_retrieved_docs,k):
  return relevant_retrieved_docs/k

def calculate_parameters(top_50_list, test_mat):
  recall5 = np.zeros((num_user,1))
  precision5 = np.zeros((num_user,1))
  recall20 = np.zeros((num_user,1))
  precision20 = np.zeros((num_user,1))
  recall50 = np.zeros((num_user,1))
  precision50 = np.zeros((num_user,1))
  # recall = np.zeros((num_user,3))
  # precision = np.zeros((num_user,3))

  total_relevant = test_mat.sum(axis=1).astype(int)
  print(total_relevant)
  invalid_user = np.where(total_relevant[:]== 0)[0]

  for u in range(num_user):
    relevant_retrieved_docs = 0
    if u in invalid_user:
      continue
    #calculation for k = 5
    for k in range(5):
      relevant_retrieved_docs = relevant_retrieved_docs + 1 if test_mat[u,int(top_50_list[u,k])] == 1 else relevant_retrieved_docs
    recall5[u,0] = recall_func(relevant_retrieved_docs,total_relevant[u])
    precision5[u,0] = precision_func(relevant_retrieved_docs,5)
    relevant_retrieved_docs = 0
    #calculation for k = 20
    for k in range(20):
      relevant_retrieved_docs = relevant_retrieved_docs + 1 if test_mat[u,int(top_50_list[u,k])] == 1 else relevant_retrieved_docs
    recall20[u,0] = recall_func(relevant_retrieved_docs,total_relevant[u])
    precision20[u,0] = precision_func(relevant_retrieved_docs,20)
    relevant_retrieved_docs = 0
    #calculation for k = 50
    for k in range(50):
      relevant_retrieved_docs = relevant_retrieved_docs + 1 if test_mat[u,int(top_50_list[u,k])] == 1 else relevant_retrieved_docs
    recall50[u,0] = recall_func(relevant_retrieved_docs,total_relevant[u])
    precision50[u,0] = precision_func(relevant_retrieved_docs, 50)


  print('Average Recall values for k = 5 is', (recall5.sum(axis = 0))/num_user)
  print('Average Recall values for k = 20 is', (recall20.sum(axis = 0))/num_user)
  print('Average Recall values for k = 50 is', (recall50.sum(axis = 0))/num_user)

  print('Average Precision values for k = 5 is', (precision5.sum(axis = 0))/num_user)
  print('Average Precision values for k = 20 is', (precision20.sum(axis = 0))/num_user)
  print('Average Precision values for k = 50 is', (precision50.sum(axis = 0))/num_user)

calculate_parameters(top_50_list, test_mat)



[15 35 13 ...  6 41 98]
Average Recall values for k = 5 is [0.03786131]
Average Recall values for k = 20 is [0.10863614]
Average Recall values for k = 50 is [0.19360902]
Average Precision values for k = 5 is [0.27301325]
Average Precision values for k = 20 is [0.21042219]
Average Precision values for k = 50 is [0.15778477]


# Part 2. Building the Collaborative Filtering Model for Implicit Feedback 

In this part, we will need to build **personalized** models instead of non-personalized models as in Part 1. We study how collaborative filtering algorithms work for recommendations with implicit feedback. The overall pipeline is the same as in Part 1. But now you need to implement a user-user collaborative filtering algorithm for recommendation. You will also evaluate your personalized models to compare them with the non-personalized one in Part 1.

In this part, you will use the same MovieLens 1M dataset, and:

* write the code to implement a user-user collaborative filtering algorithm,
* and evaluate your recommender.

First, we need to load and preprocess the experiment dataset. We still use the MovieLens 1M data from https://grouplens.org/datasets/movielens/1m/ in this part.

Then, you need to write code to implement a user-user collaborative filtering algorithm with **cosine similarity**. Although we have not discussed in class how to use collaborative filtering for implicit feedback, it is quite straightforward. 

* We first need to calculate the cosine similarity between users based on the binary feedback vectors of users; 
* Then, for a specific user $u$, we predict a preference vector for the user $u$ by weighted averaging the binary feedback vectors of N users who are most similar to $u$;
* And last, rank the movies based on the predicted preference vector of the user $u$ as recommendations. 

The predicted preference score from user $u$ to movie $i$ can be calculated as: $p_{u,i}=\frac{\sum_{u^\prime\in N}s(u,u^\prime)r_{u^\prime,i}}{\sum_{u^\prime\in N}|s(u,u^\prime)|}$, where $s(u,u^\prime)$ is the cosine similarity, and we set the size of $N$ as 10.

In the next cell, write your code to generate the ranked lists of movies by this user-user collaborative filtering recommendation algorithm for every user, store the result in a numpy array of size (#user, 50), where entry (u, k) represents the movie id that is ranked at position k in the recommendation list to user u. 

* Hint: for a user $u$, the movies user $u$ liked in the train_mat should be excluded in the top 50 recommendation list

In [60]:
# Your Code Here...
train_mat
test_mat
cosine_mat = np.zeros((num_user,num_user))
top_50_CF = np.zeros((num_user,50))
for UserID in range(num_user):
  for user_comp in range(num_user):
    if UserID != user_comp:
      cosine_mat[UserID][user_comp] = (np.dot(train_mat[UserID], train_mat[user_comp])/(np.linalg.norm(train_mat[UserID])*np.linalg.norm(train_mat[user_comp])))
# print(cosine_mat.shape)
# print(cosine_mat)

for index, user in enumerate(train_mat):
  knn_index = cosine_mat[index].argsort()[:-11:-1]
  knn_simi_score = cosine_mat[index][knn_index]
  preferance_vector = np.dot(knn_simi_score, train_mat[knn_index]) / knn_simi_score.sum()
  watched_movies = np.where(train_mat[index,:]== 1)[0]
  preferance_vector[watched_movies] = 0
  top_50_CF[index] = preferance_vector.argsort()[:-51:-1]

print(top_50_CF[0,:5])


print('\nTop 5 recommended movies for first user are:')
for i in range(5):
  print('Movie ID:', top_50_CF[0,i], 'is at', i+1, 'position')
  



[207.  10. 390. 104. 381.]

Top 5 recommended movies for first user are:
Movie ID: 207.0 is at 1 position
Movie ID: 10.0 is at 2 position
Movie ID: 390.0 is at 3 position
Movie ID: 104.0 is at 4 position
Movie ID: 381.0 is at 5 position


Last, you need to evaluate your user-user collaborative filtering algorithm by the held-out testing dataset test_mat for each user. Here, we use the same metrics recall@k and precision@k we used in Part 2c of Module 1. So you can use the same code here to calculate recall@k and precision@k for k=5, 20, 50 for each user, i.e., six metrics for every user. And please print out the average over all users for these six metrics.

In [61]:
# Calculate recall@k, precision@k with k=5, 20, 50 and print out the average over all users for these 6 metrics.
# Your Code Here...

calculate_parameters(top_50_CF, test_mat)


[15 35 13 ...  6 41 98]
Average Recall values for k = 5 is [0.07063241]
Average Recall values for k = 20 is [0.191786]
Average Recall values for k = 50 is [0.32227182]
Average Precision values for k = 5 is [0.37821192]
Average Precision values for k = 20 is [0.29107616]
Average Precision values for k = 50 is [0.22180132]


**Question (5 points):** do you observe a better result compared with models implemented in the previous part? What's reason do you think that brings this improvement?

User-user collaborative filtering give better result than non-personalized recommender because it consider the individual preferences of each user. Non-personalized recommenders recommends the item by popularity and do not consider the unique preferences of individual user for which recommendation was made.

# Part 3. Building the Matrix Factorization Model for Implicit Feedback 

Now we turn to study how matrix factorization algorithms works for recommendations with implicit feedback. The overall pipeline is the same as in Part 1 and Part 2. But now you need to implement a matrix factorization algorithm for recommendation. You will also compare your matrix factorization model to the non-personalized one in Part 1 and collaborative filtering one in Part 2.

In this part, you will use the same MovieLens 1M dataset, and:
* write the code to implement a matrix factorization algorithm,
* and evaluate your recommender.

First, let's recap the matrix factorization model introduced in class. The MF model can be mathematically represented as: 

<center>$\underset{\mathbf{P},\mathbf{Q}}{\text{min}}\,\,L=\sum_{(u,i)\in\mathcal{O}}(\mathbf{P}_u\cdot\mathbf{Q}^\top_i-r_{u,i})^2+\lambda(\lVert\mathbf{P}\rVert^2_{\text{F}}+\lVert\mathbf{Q}\rVert^2_{\text{F}})$,</center>
    
where $\mathbf{P}$ is the user latent factor matrix of size (#user, #latent); $\mathbf{Q}$ is the movie latent factor matrix of size (#movie, #latent); $\mathcal{O}$ is a user-movie pair set containing all user-movie pairs having ratings in train_mat; $r_{u,i}$ represents the rating for user u and movie i; $\lambda(\lVert\mathbf{P}\rVert^2_{\text{F}}+\lVert\mathbf{Q}\rVert^2_{\text{F}})$ is the regularization term to overcome overfitting problem, $\lambda$ is the regularization weight (a hyper-parameter manually set by developer, i.e., you), and $\lVert\mathbf{P}\rVert^2_{\text{F}}=\sum_{x}\sum_{y}(\mathbf{P}_{x,y})^2$, $\lVert\mathbf{Q}\rVert^2_{\text{F}}=\sum_{x}\sum_{y}(\mathbf{Q}_{x,y})^2$. Such an L function is called the **loss function** for the matrix factorization model. The goal of training an MF model is to find appropriate $\mathbf{P}$ and $\mathbf{Q}$ to minimize the loss L.

Although we have not discussed in class how to use matrix factorization algorithm for implicit feedback, it is quite straightforward. The main idea is the same as the MF model for explicit ratings. But instead of predicting explicit ratings, here the MF is to predict binary ratings based on which movies are ranked for users.

The only challenge now is that in an implicit feedback dataset, there is only positive signal (i.e., '1' in the train_mat) without negative signal. Hence, to let our MF work for this implicit feedback, a simple but powerful method -- random negative sampling -- is adopted. The main idea is that in each training epoch, we randomly sample user-movie pairs without positive feedback in train_mat (user-movie pairs with '0' in train_mat) to be negative feedback, and mix them with positive feedback as the training data to train the MF model. The **negative sampling method is already provided**.

In the Next cell, you need to complete the MF_implicit class. There are five functions in this class: 

* 'init' (**provided**) is to initialize the variables the MF class needs, which takes 5 inputs: train_mat, test_mat, latent, lr, and reg. 'train_mat' and 'test_mat' are the corresponfing training and testing matrices we have. 'latent' represents the latent dimension we set for the MF model. 'lr' represents the learning rate, i.e., the update step in each optimization iteration, default is 0.01. 'reg' represents the regularization weight, i.e., the $\lambda$ in the MF formulation. 

* 'negative_sampling' (**provided**) is to do the random negative sampling to generate negative signals for training the MF. It returns a list of users and a list of movies as the training samples, mixing positive and negative user-movie pairs.

* 'test' (**provided**) is to evaluate the trained MF on test_mat and print out recall@k and precision@k. 

* 'predict' (**need to be completed**) is to generate the ranked lists of movies by the trained MF for every user, store the ranking result (named 'recommendation') in a numpy array of size (#user, 50), where entry (u, k) represents the movie id that is ranked at position k in the recommendation list to user u. Return the 'recommendation' variable. 

* 'train' (**need to be completed**) is to train the MF model. There is only one input to this function: an int variable 'epoch' to indicate how many epochs for training the model. The main logic of this function is the same as the one you already implemented in Part 1a. The main body of this function should be a loop for 'epoch' iterations. In each iteration, following the algorithm to update the MF model:

        1. Call 'negative_sampling()' to generate a list of users and a list of movies mixing positive and negative user-movie pairs as training samples.
        2. Randomly shuffle training user-movie pairs  (i.e., user-movie pairs from step 1)
        2. Have an inner loop to iterate each user-movie pair:
                a. given a user-movie pair (u,i), update the user latent factor and movie latent factor by gradient decsent:    
<center>$\mathbf{P}_u=\mathbf{P}_u-\gamma [2(\mathbf{P}_u\cdot\mathbf{Q}_i^\top-r_{u,i})\cdot\mathbf{Q}_i+2\lambda\mathbf{P}_u]$</center>    
<center>$\mathbf{Q}_i=\mathbf{Q}_i-\gamma [2(\mathbf{P}_u\cdot\mathbf{Q}_i^\top-r_{u,i})\cdot\mathbf{P}_u+2\lambda\mathbf{Q}_i]$</center>    
<center>where $\mathbf{P}_u$ and $\mathbf{Q}_i$ are row vectors of size (1, #latent), $\gamma$ is learning rate (default is 0.01), $\lambda$ is regularization weight, and $r_{u,i}$ now takes binary value.</center>
        
        3. After iterating over all user-movie pairs, call 'test()' to evaluate the current MF model.

**Note: you are not supposed to delete or modify the provided code.**

**Note: for this part, it is necessary to read and understand the provided code, because you will need to call the provided functions in your code.**

In [62]:
class MF_implicit:
    def __init__(self, train_mat, test_mat, latent=5, lr=0.01, reg=0.01):
        self.train_mat = train_mat  # the training rating matrix of size (#user, #movie)
        self.test_mat = test_mat  # the training rating matrix of size (#user, #movie)
        
        self.latent = latent  # the latent dimension
        self.lr = lr  # learning rate
        self.reg = reg  # regularization weight, i.e., the lambda in the objective function
        
        self.num_user, self.num_movie = train_mat.shape
        
        self.sample_user, self.sample_movie = self.train_mat.nonzero()  # get the user-movie paris having ratings in train_mat
        self.num_sample = len(self.sample_user)  # the number of user-movie pairs having ratings in train_mat

        self.user_test_like = []
        for u in range(self.num_user):
            self.user_test_like.append(np.where(self.test_mat[u, :] > 0)[0])

        self.P = np.random.random((self.num_user, self.latent))  # latent factors for users, size (#user, self.latent), randomly initialized
        self.Q = np.random.random((self.num_movie, self.latent))  # latent factors for users, size (#movie, self.latent), randomly initialized
        
    def negative_sampling(self):
        negative_movie = np.random.choice(np.arange(self.num_movie), size=(len(self.sample_user)), replace=True)
        true_negative = self.train_mat[self.sample_user, negative_movie] == 0
        negative_user = self.sample_user[true_negative]
        negative_movie = negative_movie[true_negative]
        return np.concatenate([self.sample_user, negative_user]), np.concatenate([self.sample_movie, negative_movie])

    def train(self, epoch=20):
        """
        Goal: Write your code to train your matrix factorization model for epoch iterations in this function
        Input: epoch -- the number of training epoch 
        """
        for ep in range(epoch):
            """ 
            Write your code here to implement the training process for one epoch, 
            at the end of each epoch, run self.test() to evaluate current version of MF.
            """
            user_indices, movie_indices = self.negative_sampling()
            user_movie_pairs = list(zip(user_indices, movie_indices))

            np.random.shuffle(user_movie_pairs)
            
            for user, movie in user_movie_pairs:
                r_user_movie = self.train_mat[user, movie]
                estimate = np.dot(self.P[user], self.Q[movie].T) - r_user_movie
                self.P[user] = self.P[user] - self.lr * 2 * (estimate * self.Q[movie] + self.reg * self.P[user])
                self.Q[movie] = self.Q[movie] - self.lr * 2 * (estimate * self.P[user] + self.reg * self.Q[movie])
            
            self.test()
            
            """
            End of your code for this function
            """
            
    def predict(self):
        """
        Write your code here to implement the prediction function, which generates the ranked lists of movies 
        by the trained MF for every user, store the result (named 'recommendation') in a numpy array of size (#user, 50), where entry (u, k) 
        represents the movie id that is ranked at position k in the recommendation list to user u. Return the 'recommendation' variable. 
        """
        prediction_mat = np.matmul(self.P, self.Q.T)
        recommendation = []
        for u in range(self.num_user):
            scores = prediction_mat[u]
            train_like = np.where(train_mat[u, :] > 0)[0]
            scores[train_like] = -9999
            top50_iid = np.argpartition(scores, -50)[-50:]
            top50_iid = top50_iid[np.argsort(scores[top50_iid])[-1::-1]]
            recommendation.append(top50_iid)
        recommendation = np.array(recommendation)
        
        
        """
        End of your code for this function
        """
        return recommendation
    
    def test(self):
        recommendation = self.predict()

        recalls = np.zeros(3)
        precisions = np.zeros(3)
        user_count = 0.

        for u in range(self.num_user):
            test_like = self.user_test_like[u]
            test_like_num = len(test_like)
            if test_like_num == 0:
                continue
            rec = recommendation[u, :]
            hits = np.zeros(3)
            for k in range(50):
                if rec[k] in test_like:
                    if k < 50:
                        hits[2] += 1
                        if k < 20:
                            hits[1] += 1
                            if k < 5:
                                hits[0] += 1
            recalls[0] += (hits[0] / test_like_num)
            recalls[1] += (hits[1] / test_like_num)
            recalls[2] += (hits[2] / test_like_num)
            precisions[0] += (hits[0] / 5.)
            precisions[1] += (hits[1] / 20.)
            precisions[2] += (hits[2] / 50.)
            user_count += 1

        recalls /= user_count
        precisions /= user_count

        print('recall@5\t[%.6f],\t||\t recall@20\t[%.6f],\t||\t recall@50\t[%.6f]' % (recalls[0], recalls[1], recalls[2]))
        print('precision@5\t[%.6f],\t||\t precision@20\t[%.6f],\t||\t precision@50\t[%.6f]' % (precisions[0], precisions[1], precisions[2]))
        print('')

Now, run the next cell to build and train your implemented MF model for implicit feedback. The expected time used for training one epoch is 20s to 2 min.

In [63]:
mf_implicit = MF_implicit(train_mat, test_mat, latent=5, lr=0.01, reg=0.0001)
mf_implicit.train(epoch=20)

recall@5	[0.012573],	||	 recall@20	[0.053551],	||	 recall@50	[0.130694]
precision@5	[0.135000],	||	 precision@20	[0.133626],	||	 precision@50	[0.123311]

recall@5	[0.022999],	||	 recall@20	[0.079590],	||	 recall@50	[0.161676]
precision@5	[0.207583],	||	 precision@20	[0.175017],	||	 precision@50	[0.143387]

recall@5	[0.030318],	||	 recall@20	[0.089967],	||	 recall@50	[0.173460]
precision@5	[0.238311],	||	 precision@20	[0.186697],	||	 precision@50	[0.148815]

recall@5	[0.025698],	||	 recall@20	[0.090851],	||	 recall@50	[0.175967]
precision@5	[0.219636],	||	 precision@20	[0.192368],	||	 precision@50	[0.150758]

recall@5	[0.030231],	||	 recall@20	[0.092757],	||	 recall@50	[0.176716]
precision@5	[0.246291],	||	 precision@20	[0.197053],	||	 precision@50	[0.154728]

recall@5	[0.026619],	||	 recall@20	[0.087355],	||	 recall@50	[0.179300]
precision@5	[0.226556],	||	 precision@20	[0.196846],	||	 precision@50	[0.163692]

recall@5	[0.027956],	||	 recall@20	[0.094028],	||	 recall@50	[0.185375]
prec

**Question (5 points):** do you observe a better result compared with models implemented in previous two parts? What's reason do you think that brings this improvement?

Usually MF should give better recall and precision as compared to User-user collaborative filtering due to following reason:

* It can handle large-scale data as it convert it into smaller matrices of user and item latent factors. This makes the computation more efficient and faster.
* It can make recommendations for new users or items based on their latent factors which it learn from existing user-item interactions.
* The model could easily capture and learn the underlying patterns in the data which would be hard for methods like user-user CF.

But we can observe above that the recall and precision values achieved are less as compared to the user-user CF. This could be beacuse of following reasons:

* We can see that the used user-movie matrix is approx 89% empty and is sparse so it can be difficult for MF to learn meaningful representations of users and items. In such cases, user-user collaborative filtering can be a better option as it relies on the similarity between users.

* Currently the model is using implicit feedback data which do not provide much details about item and is inherently noisy.

* Inadequate hypermeter tuning - currently we are using the given hyperparameters values - number of latent factors = 5, Learning rate = 0.01 and regularization factor 0.001. It might be possible that model is getting underfit or overfit for these particular hyperparameters and require further tuning.

* It might be a case that data is highly skewed towards certain items or certain types of users which makes it diffcult for model to make accurate recommendations for a diverse set of users and items.

# Part 4: Building the Bayesian Personalized Ranking (BPR) Model with Implicit Feedback 

In this first part, you will need to build a **Bayesian Personalized Ranking (BPR)** model. You will also evaluate your BPR model to compare with the non-personalized one in Part 1, neighborhood-based collaborative filtering model in Part 2, and matrix factorization model in Part 3. 

For this part, we will:

* build a BPR model,
* and evaluate your recommender.

We still use the MovieLens 1M data from https://grouplens.org/datasets/movielens/1m/ in this part, and use the same loading and preprocessing process.

First, let's implement the Bayesian Personalized Ranking model introduced in class. The BPR model can be mathematically represented as: 

<center>$\underset{\mathbf{P},\mathbf{Q}}{\text{max}}\,\,L=\sum_{(u,i,j)\in\mathcal{O}}\sigma(\mathbf{P}_u\cdot\mathbf{Q}^\top_i-\mathbf{P}_u\cdot\mathbf{Q}^\top_j)+\lambda(\lVert\mathbf{P}\rVert^2_{\text{F}}+\lVert\mathbf{Q}\rVert^2_{\text{F}})$,</center>
    
where $\mathbf{P}$ is the user latent factor matrix of size (#user, #latent); $\mathbf{Q}$ is the movie latent factor matrix of size (#movie, #latent); $\sigma(\cdot)$ is the Sigmoid function; $\mathcal{O}$ is a (user, positive movie, negative movie) tuple set, and each tuple $(u,i,j)$ is a training sample with user $u$ and a posotive movie $i$ with value 1 in train_mat and a negative movie $j$ with value 0 in train_mat; $\lambda(\lVert\mathbf{P}\rVert^2_{\text{F}}+\lVert\mathbf{Q}\rVert^2_{\text{F}})$ is the regularization term to overcome overfitting problem, $\lambda$ is the regularization weight (a hyper-parameter manually set by the developer, i.e., you), and $\lVert\mathbf{P}\rVert^2_{\text{F}}=\sum_{x}\sum_{y}(\mathbf{P}_{x,y})^2$, $\lVert\mathbf{Q}\rVert^2_{\text{F}}=\sum_{x}\sum_{y}(\mathbf{Q}_{x,y})^2$. Such an L function is called the **loss function** for the matrix factorization model. The goal of training a BPR model is to find appropriate $\mathbf{P}$ and $\mathbf{Q}$ to **maximize** the loss L.

To implement such a BPR model, here we will write a Python class for the model. Similar to the implicit_MF model you have already implemented in Part 3, there are five functions in this BPR class: 

* The 'init' function (**provided**) is to initialize the variables the BPR class needs, which takes 5 inputs: train_mat, test_mat, latent, lr, and reg. 'train_mat' and 'test_mat' are the corresponfing training and testing matrices we have. 'latent' represents the latent dimension we set for the BPR model. 'lr' represents the learning rate, i.e., the update step in each optimization iteration, default is 0.01. 'reg' represents the regularization weight, i.e., the $\lambda$ in the BPR formulation.

* 'negative_sampling' (**provided**) is to do the random negative sampling to generate negative signals for training the BPR. It returns a list of users, a list of positive movies of these users, and a list of negative movies of these user. These three lists form the training set $\mathcal{O}$.

* 'test' (**provided**) is to evaluate the trained MF on test_mat and print out recall@k and precision@k. 

* 'predict' (**provided**) is to generates the ranked lists of movies by the trained MF for every user, store the ranking result (named 'recommendation') in a numpy array of size (#user, 50), where entry (u, k) represents the movie id that is ranked at position k in the recommendation list to user u. Return the 'recommendation' variable. 

* 'train' (**need to be completed**) is to train the BPR model. There is only one input to this function: an int variable 'epoch' to indicate how many epochs for training the model. The main process of one training epoch is:

        1. Call 'negative_sampling()' to generate training samples: a list of users, a list of positive movies of these users, and a list of negative movies of these users.
        2. Randomly shuffle training samples from step 1.
        3. Have an inner loop to iterate each (user, positive movie, negative movie) tuple in the shuffled training samples:
                a. given the current training tuple -- a user u, a positive movie i, and a negative movie j, update the user latent factor and movie latent factor by gradient decsent:
<center>$\mathbf{P}_u=\mathbf{P}_u - \gamma[-\frac{e^{-\widehat{x}}}{1 + e^{-\widehat{x}}} (\mathbf{Q}_i - \mathbf{Q}_j) + \lambda\mathbf{P}_u]$</center>
<center>$\mathbf{Q}_i=\mathbf{Q}_i - \gamma[-\frac{e^{-\widehat{x}}}{1 + e^{-\widehat{x}}} \mathbf{P}_U + \lambda\mathbf{Q}_i]$</center>
<center>$\mathbf{Q}_j=\mathbf{Q}_j - \gamma[\frac{e^{-\widehat{x}}}{1 + e^{-\widehat{x}}} \mathbf{P}_U + \lambda\mathbf{Q}_j]$</center>   
<center>where $\mathbf{P}_u$, $\mathbf{Q}_i$, and $\mathbf{Q}_j$ are of size (1, #latent), $\gamma$ is learning rate (default is 0.01), $\lambda$ is regularization weight, and $\widehat{x}=\mathbf{P}_u\cdot\mathbf{Q}^\top_i-\mathbf{P}_u\cdot\mathbf{Q}^\top_j$.</center>
        
        3. After iterate all training samples, call 'test()' to evaluate the current BPR model.


In the next cell, you will need to fill in the 'train' function based on the description above. 

*Hint that similar to the training process of MF in Part 3, in each update iteration, you need to use the old P and Q from last update iteration to update P and Q in current update iteration.

**NOTE that you should not delete or modify the provided code.**

In [64]:
import math
class BPR:
    def __init__(self, train_mat, test_mat, latent=5, lr=0.01, reg=0.01):
        self.train_mat = train_mat  # the training rating matrix of size (#user, #movie)
        self.test_mat = test_mat  # the training rating matrix of size (#user, #movie)
        
        self.latent = latent  # the latent dimension
        self.lr = lr  # learning rate
        self.reg = reg  # regularization weight, i.e., the lambda in the objective function
        
        self.num_user, self.num_movie = train_mat.shape
        
        self.positive_user, self.positive_movie = self.train_mat.nonzero()  # get the user-movie paris having ratings in train_mat

        self.user_test_like = []
        for u in range(self.num_user):
            self.user_test_like.append(np.where(self.test_mat[u, :] > 0)[0])

        self.P = np.random.random((self.num_user, self.latent))  # latent factors for users, size (#user, self.latent), randomly initialized
        self.Q = np.random.random((self.num_movie, self.latent))  # latent factors for users, size (#movie, self.latent), randomly initialized
        
    def negative_sampling(self): 
        # do the negative sampling for each of the positive user-movie pair. Here we set negative sampling rate as 2, 
        # i.e., for each positive user-movie pair, randomly sample two negative movies. 
        # This function returns final training data: a user list, a positive movie list, and a negative movie list
        sample_user = np.tile(self.positive_user, 2)
        sample_pos_movie = np.tile(self.positive_movie, 2)
        sample_neg_movie = np.random.choice(np.arange(self.num_movie), size=(len(sample_user)), replace=True)
        true_negative = self.train_mat[sample_user, sample_neg_movie] == 0
        return sample_user[true_negative], sample_pos_movie[true_negative], sample_neg_movie[true_negative]

    def train(self, epoch=20):
        """
        Goal: Write your code to train your matrix factorization model for epoch iterations in this function
        Input: epoch -- the number of training epoch 
        """
        for ep in range(epoch):
            """ 
            Write your code here to implement the training process for one epoch, 
            at the end of each epoch, run self.test() to evaluate current version of BPR.
            """
            user_list, pos_list, neg_list = self.negative_sampling()
            uij_samples = list(zip(user_list, pos_list, neg_list))
            np.random.shuffle(uij_samples)
            for user, pos_item, neg_item in uij_samples:
                
                pos_latent = self.Q[pos_item]
                neg_latent = self.Q[neg_item]
                user_latent = self.P[user]
                x_hat = np.dot(user_latent, pos_latent - neg_latent)
                
                sigmoid_x_hat = math.exp(-x_hat) / (1 + math.exp(-x_hat))

                self.P[user] -= self.lr * (-sigmoid_x_hat * (pos_latent - neg_latent) + self.reg * user_latent)
                self.Q[pos_item] -= self.lr * (-sigmoid_x_hat * user_latent + self.reg * pos_latent)
                self.Q[neg_item] -= self.lr * (sigmoid_x_hat * user_latent + self.reg * neg_latent)

            self.test()
            """
            End of your code for this function
            """
            
    def predict(self):
        # The prediction function, which generates the ranked lists of movies 
        # by the trained BPR for every user, store the result (named 'recommendation') in a numpy array of size (#user, 50), where entry (u, k) 
        # represents the movie id that is ranked at position k in the recommendation list to user u. Return the 'recommendation' variable. 
        prediction_mat = np.matmul(self.P, self.Q.T)
        recommendation = []
        for u in range(self.num_user):
            scores = prediction_mat[u]
            train_like = np.where(train_mat[u, :] > 0)[0]
            scores[train_like] = -9999
            top50_iid = np.argpartition(scores, -50)[-50:]
            top50_iid = top50_iid[np.argsort(scores[top50_iid])[-1::-1]]
            recommendation.append(top50_iid)
        recommendation = np.array(recommendation)
        return recommendation
    
    def test(self):
        recommendation = self.predict()

        recalls = np.zeros(3)
        precisions = np.zeros(3)
        user_count = 0.

        for u in range(self.num_user):
            test_like = self.user_test_like[u]
            test_like_num = len(test_like)
            if test_like_num == 0:
                continue
            rec = recommendation[u, :]
            hits = np.zeros(3)
            for k in range(50):
                if rec[k] in test_like:
                    if k < 50:
                        hits[2] += 1
                        if k < 20:
                            hits[1] += 1
                            if k < 5:
                                hits[0] += 1
            recalls[0] += (hits[0] / test_like_num)
            recalls[1] += (hits[1] / test_like_num)
            recalls[2] += (hits[2] / test_like_num)
            precisions[0] += (hits[0] / 5.)
            precisions[1] += (hits[1] / 20.)
            precisions[2] += (hits[2] / 50.)
            user_count += 1

        recalls /= user_count
        precisions /= user_count

        print('recall@5\t[%.6f],\t||\t recall@20\t[%.6f],\t||\t recall@50\t[%.6f]' % (recalls[0], recalls[1], recalls[2]))
        print('precision@5\t[%.6f],\t||\t precision@20\t[%.6f],\t||\t precision@50\t[%.6f]' % (precisions[0], precisions[1], precisions[2]))
        print('')

Now, let's train a BPR model based on your implementation. The code is provided, you just need to execute the next cell. The expectations are: 

* first, the code can be successfully executed without error; 
* and second, the recall@k and precision@k on **test_mat** of each training epoch should be printed out for all 20 epochs.


* Hint: the expected time used for training is around 1 minute to 5 minutes per training epoch.

In [65]:
bpr = BPR(train_mat, test_mat, latent=5, lr=0.01, reg=0.001)
bpr.train(epoch=20)

recall@5	[0.034968],	||	 recall@20	[0.105035],	||	 recall@50	[0.188376]
precision@5	[0.258477],	||	 precision@20	[0.206465],	||	 precision@50	[0.154798]

recall@5	[0.034815],	||	 recall@20	[0.105514],	||	 recall@50	[0.189177]
precision@5	[0.260166],	||	 precision@20	[0.206283],	||	 precision@50	[0.155222]

recall@5	[0.035541],	||	 recall@20	[0.105341],	||	 recall@50	[0.188762]
precision@5	[0.263278],	||	 precision@20	[0.206821],	||	 precision@50	[0.155361]

recall@5	[0.035530],	||	 recall@20	[0.105910],	||	 recall@50	[0.190542]
precision@5	[0.265099],	||	 precision@20	[0.207980],	||	 precision@50	[0.157023]

recall@5	[0.036079],	||	 recall@20	[0.106274],	||	 recall@50	[0.192402]
precision@5	[0.267384],	||	 precision@20	[0.210248],	||	 precision@50	[0.159563]

recall@5	[0.036504],	||	 recall@20	[0.107039],	||	 recall@50	[0.193903]
precision@5	[0.270728],	||	 precision@20	[0.212202],	||	 precision@50	[0.162331]

recall@5	[0.036495],	||	 recall@20	[0.108971],	||	 recall@50	[0.197809]
prec

**Question (5 points):** do you observe a better result compared with models implemented in previous three parts? What's reason do you think that brings this improvement?

BPR is giving better recall and precision as compared to MF due to following reason:

* BPR is designed for implicit feedback. Absence of feedback indicates that the user is not interested in the item. But MF works better with explicit feedback where the user provides explicit ratings or feedback for the items. 

* In BPR the ranking of items is based on the pairwise comparisons of items by the user. This approach is more flexible than the approach used in MF which may not capture the complex relationships between the user and items.


But we can observe above that the BPR recall and precision values achieved are less as compared to the user-user CF. This could be beacuse of following reasons:

* We can see that the used user-movie matrix is approx 89% empty and is sparse so it can be difficult for BPR to learn meaningful representations of users and items. In such cases, user-user collaborative filtering can be a better option as it relies on the similarity between users.

* Inadequate hypermeter tuning - currently we are using the given hyperparameters values - number of latent factors = 5, Learning rate = 0.01 and regularization factor 0.001. It might be possible that model is getting underfit or overfit for these particular hyperparameters and require further tuning.

* It might be a case that data is highly skewed towards certain items or certain types of users which makes it diffcult for model to make accurate recommendations for a diverse set of users and items.

# Part 5. Cool Extension! 

Just like in the first homework, now is your chance to try an interesting extension. 

I am adding user bias and item bias parameters to BPR. BPR will learn the user bias and item bias parameters on its own instead of explicitly coding it as we do in baseline estimate. 

Also I am using the movie popularity score and using it as the initial values for item bias which will add more weight to the movies that are famous and give less weightage to movies that are never watched during start and will help model to converge faster.

In [70]:
class BPRwith_item_popularity_user_item_bias:
    def __init__(self, train_mat, test_mat, latent=5, lr=0.01, reg=0.01):
        self.train_mat = train_mat  # the training rating matrix of size (#user, #movie)
        self.test_mat = test_mat  # the training rating matrix of size (#user, #movie)
        
        self.latent = latent  # the latent dimension
        self.lr = lr  # learning rate
        self.reg = reg  # regularization weight, i.e., the lambda in the objective function
        
        self.num_user, self.num_movie = train_mat.shape
        
        self.positive_user, self.positive_movie = self.train_mat.nonzero()  # get the user-movie paris having ratings in train_mat

        self.user_test_like = []
        for u in range(self.num_user):
            self.user_test_like.append(np.where(self.test_mat[u, :] > 0)[0])

        self.popularity = train_mat.sum(axis=0).astype(float)
        z = np.where(self.popularity == 0)[0] 
        self.popularity[z] = 0.001
        self.popularity = 1/np.log(1+self.popularity)
        
        self.bu = np.random.random((num_user,1))
        self.bi = self.popularity
        self.P = np.random.random((self.num_user, self.latent))  # latent factors for users, size (#user, self.latent), randomly initialized
        self.Q = np.random.random((self.num_movie, self.latent))  # latent factors for users, size (#movie, self.latent), randomly initialized
        
    def negative_sampling(self): 
        # do the negative sampling for each of the positive user-movie pair. Here we set negative sampling rate as 2, 
        # i.e., for each positive user-movie pair, randomly sample two negative movies. 
        # This function returns final training data: a user list, a positive movie list, and a negative movie list
        sample_user = np.tile(self.positive_user, 2)
        sample_pos_movie = np.tile(self.positive_movie, 2)
        sample_neg_movie = np.random.choice(np.arange(self.num_movie), size=(len(sample_user)), replace=True)
        true_negative = self.train_mat[sample_user, sample_neg_movie] == 0
        return sample_user[true_negative], sample_pos_movie[true_negative], sample_neg_movie[true_negative]

    def train(self, epoch=20):
        """
        Goal: Write your code to train your matrix factorization model for epoch iterations in this function
        Input: epoch -- the number of training epoch 
        """
        
        for ep in range(epoch):
            """ 
            Write your code here to implement the training process for one epoch, 
            at the end of each epoch, run self.test() to evaluate current version of BPR.
            """
            user_list, pos_list, neg_list = self.negative_sampling()
            uij_samples = list(zip(user_list, pos_list, neg_list))
            np.random.shuffle(uij_samples)
            for user, pos_item, neg_item in uij_samples:
                
                pos_latent = self.Q[pos_item]
                neg_latent = self.Q[neg_item]
                user_latent = self.P[user]
                x_hat = np.dot(user_latent, pos_latent - neg_latent) + self.bu[user] + self.bi[pos_item]
                
                sigmoid_x_hat = np.exp(-x_hat) / (1 + np.exp(-x_hat)) 

                self.P[user] -= self.lr * (-sigmoid_x_hat * (pos_latent - neg_latent) + self.reg * user_latent) 
                self.Q[pos_item] -= self.lr * (-sigmoid_x_hat * user_latent + self.reg * pos_latent)
                self.Q[neg_item] -= self.lr * (sigmoid_x_hat * user_latent + self.reg * neg_latent)
                self.bu[user] -= self.lr * (-sigmoid_x_hat + self.reg * self.bu[user])
                self.bi[pos_item] -= self.lr * (-sigmoid_x_hat + self.reg * self.bi[pos_item])

            self.test()
            """
            End of your code for this function
            """
            
    def predict(self):
        # The prediction function, which generates the ranked lists of movies 
        # by the trained BPR for every user, store the result (named 'recommendation') in a numpy array of size (#user, 50), where entry (u, k) 
        # represents the movie id that is ranked at position k in the recommendation list to user u. Return the 'recommendation' variable. 
        prediction_mat = np.matmul(self.P, self.Q.T)
        recommendation = []
        for u in range(self.num_user):
            scores = prediction_mat[u]
            train_like = np.where(train_mat[u, :] > 0)[0]
            scores[train_like] = -9999
            top50_iid = np.argpartition(scores, -50)[-50:]
            top50_iid = top50_iid[np.argsort(scores[top50_iid])[-1::-1]]
            recommendation.append(top50_iid)
        recommendation = np.array(recommendation)
        return recommendation
    
    def test(self):
        recommendation = self.predict()

        recalls = np.zeros(3)
        precisions = np.zeros(3)
        user_count = 0.

        for u in range(self.num_user):
            test_like = self.user_test_like[u]
            test_like_num = len(test_like)
            if test_like_num == 0:
                continue
            rec = recommendation[u, :]
            hits = np.zeros(3)
            for k in range(50):
                if rec[k] in test_like:
                    if k < 50:
                        hits[2] += 1
                        if k < 20:
                            hits[1] += 1
                            if k < 5:
                                hits[0] += 1
            recalls[0] += (hits[0] / test_like_num)
            recalls[1] += (hits[1] / test_like_num)
            recalls[2] += (hits[2] / test_like_num)
            precisions[0] += (hits[0] / 5.)
            precisions[1] += (hits[1] / 20.)
            precisions[2] += (hits[2] / 50.)
            user_count += 1

        recalls /= user_count
        precisions /= user_count

        print('recall@5\t[%.6f],\t||\t recall@20\t[%.6f],\t||\t recall@50\t[%.6f]' % (recalls[0], recalls[1], recalls[2]))
        print('precision@5\t[%.6f],\t||\t precision@20\t[%.6f],\t||\t precision@50\t[%.6f]' % (precisions[0], precisions[1], precisions[2]))
        print('')

In [67]:
cool_extension = BPRwith_item_popularity_user_item_bias(train_mat, test_mat, latent=5, lr=0.02, reg=0.001)
cool_extension.train(epoch=20)

recall@5	[0.026951],	||	 recall@20	[0.086781],	||	 recall@50	[0.164726]
precision@5	[0.217517],	||	 precision@20	[0.177873],	||	 precision@50	[0.139139]

recall@5	[0.028418],	||	 recall@20	[0.088146],	||	 recall@50	[0.166548]
precision@5	[0.226192],	||	 precision@20	[0.180157],	||	 precision@50	[0.140639]

recall@5	[0.029808],	||	 recall@20	[0.090299],	||	 recall@50	[0.169363]
precision@5	[0.233742],	||	 precision@20	[0.184685],	||	 precision@50	[0.142391]

recall@5	[0.029839],	||	 recall@20	[0.092974],	||	 recall@50	[0.170933]
precision@5	[0.235828],	||	 precision@20	[0.188303],	||	 precision@50	[0.143142]

recall@5	[0.031432],	||	 recall@20	[0.094359],	||	 recall@50	[0.172857]
precision@5	[0.243609],	||	 precision@20	[0.190828],	||	 precision@50	[0.144573]

recall@5	[0.032981],	||	 recall@20	[0.096859],	||	 recall@50	[0.175545]
precision@5	[0.251854],	||	 precision@20	[0.193990],	||	 precision@50	[0.145851]

recall@5	[0.032678],	||	 recall@20	[0.098962],	||	 recall@50	[0.177379]
prec

# Collaboration Declarations

** You should fill out your collaboration declarations here.**

**Reminder:** You are expected to complete each homework independently. Your solution should be written by you without the direct aid or help of anyone else. However, we believe that collaboration and team work are important for facilitating learning, so we encourage you to discuss problems and general problem approaches (but not actual solutions) with your classmates. You may post on Piazza, search StackOverflow, etc. But if you do get help in this way, you must inform us by filling out the Collaboration Declarations at the bottom of this notebook.

Example: I found helpful code on stackoverflow at https://stackoverflow.com/questions/11764539/writing-fizzbuzz that helped me solve Problem 2.